In [48]:
import requests
def get_data(page, per_page, filtering = False):
    
    url =  f"https://www.plateapr.com/wp-json/wp/v2/posts?page={page}&per_page={per_page}"
    if filtering:
        url =  url+"&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type"

    print(f"Requesting... {url}")
    result = requests.get(url)
    if result.ok:
        print(f"Sucess request... {url}")
    else:
        print(url)
        raise Exception(
            f"Request failed: {result.text}")
    return result

In [110]:
per_page = 100
page = 1

there_are_more_pages = True
articles = []
while(there_are_more_pages):
    try:
        result = get_data(page, per_page, True)
        for article in result.json():
            articles.append(article)
        page = page + 1
    except:
        there_are_more_pages = False
        break

    

Requesting... https://www.plateapr.com/wp-json/wp/v2/posts?page=1&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Sucess request... https://www.plateapr.com/wp-json/wp/v2/posts?page=1&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Requesting... https://www.plateapr.com/wp-json/wp/v2/posts?page=2&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Sucess request... https://www.plateapr.com/wp-json/wp/v2/posts?page=2&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Requesting... https://www.plateapr.com/wp-json/wp/v2/posts?page=3&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
https://www.plateapr.com/wp-json/wp/v2/posts?page=3&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fi

In [147]:
import re
CLEANR = re.compile('<.*?>') 

def remove_html(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext


def remove_characters(raw_string):
    
    cleantext = re.sub('&#[0-9][0-9][0-9][0-9];', '', raw_string)
    cleantext = re.sub('&#[0-9][0-9][0-9];', '', cleantext)
    cleantext = re.sub('&#[0-9][0-9];', '', cleantext)
    cleantext = re.sub('&nbsp;', '', cleantext)
    cleantext = re.sub('&amp;', '&', cleantext)
    cleantext = re.sub('// ', '', cleantext)
    cleantext = cleantext.replace("|", "")
    cleantext = cleantext.replace("&#8217;", "")
    cleantext = cleantext.replace("“", "")
    cleantext = cleantext.replace("”", "")
    cleantext = cleantext.replace("’", "")
    cleantext = cleantext.replace("#", "")
    cleantext = cleantext.replace("-", " ")
    cleantext = cleantext.replace("ü", "u")

    
    return cleantext

def remove_spacing(raw_string):
    raw_string = raw_string.replace("\n", " ")
    return re.sub("\s\s+", " ", raw_string)


def remove_emojis(data):

    emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                  "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def remove_dollars(data):
    data = data.replace("$ ", "")
    data = data.replace("$", "")

    return  data

def remove_at_tags(data):
    cleantext=  re.sub(r'\s*\(@.+?\)\s*', ' ', data)
    cleantext = cleantext.replace("(", "")
    cleantext = cleantext.replace(")", "")
    return cleantext

def remove_instagram_text(data):
    return data.replace("View this post on Instagram A post shared by ", "")

def clean_article(article):
    art = dict()
    art['title'] = article['title']['rendered']
    art['content'] =  remove_instagram_text(remove_at_tags(remove_dollars(remove_emojis(remove_spacing(remove_characters(remove_html(article['content']['rendered']))))).strip()))
    art['link'] = article['link']
    art['date'] = article['date']
    art['author'] = article['author']
    art['type'] = article['type']

    return art
    

In [ ]:
cleaned = [clean_article(a) for a in articles]
cleaned[10]

{'title': 'Nueve panaderías artesanales alrededor de la isla para saciar tus antojos',
 'content': 'No hay nada mejor que un pan bien tostado, con mantequilla y café. Ya bien lo dice el refrán donde hubo pan, migajas quedan. Pero, cuando pruebes las delicias que tienen para ofrecer estas nueves panaderías artesanales, no quedará ni una sola migaja en tu plato. 1 El Horno de Pane 380 Calle Juan Calaf, San Juan, 00918 coffee shops panaderías Parking Postres Repostería El Horno de Pane En San Juan se encuentra esta panadería artesanal que ofrece una variedad de panes, postres, sándwiches, desayunos, café y tés. Inspirada en las panaderías tradicionales de los barrios de Europa, buscan que cada mañana sus panes y postres recién horneados y frescos reúnan a la familia en la mesa. ¡Tal y como es costumbre en Europa! Por eso su meta es que, en cada visita, te sientas en comunidad y que disfrutes de un pan acabado de salir del horno. Recomendación: Elige entre un clásico francés como el Baguet

In [141]:
import pandas as pd
data = pd.DataFrame.from_records(cleaned)
data.to_csv('./data/wp_posts.csv', index=False)

# Further Clean Up

In [155]:
to_clean = pd.read_csv('./data/text-similarity-documents.csv')

In [156]:
to_clean['content']= [remove_instagram_text(a) for a in to_clean['content']]

In [157]:
import pandas as pd
data = pd.DataFrame.from_records(to_clean)
data.to_csv('./data/wp_posts_cleaned.csv', index=False)